# Deep learning to predict charges of peptides

### Go beyond MS2/RT prediction for DIA search

### Prepare the data

In [1]:
import os

def get_msms_txt_list(dir, max_file_num=100000):
    msms_txt_list = []
    for msms_txt in os.listdir(dir):
        msms_txt_list.append(os.path.join(dir,msms_txt))
        if len(msms_txt_list) >= max_file_num:
            break
    return msms_txt_list

train_txt_list = get_msms_txt_list("../test_data/ProteomeKindom/train/", max_file_num=10)
test_txt_list = get_msms_txt_list("../test_data/ProteomeKindom/test/", max_file_num=10)


In [2]:
import pandas as pd
from alphabase.psm_reader import MaxQuantReader

def load_psm_df(msms_txt_list):
    psm_df_list = [
        MaxQuantReader().import_file(msms_txt) for msms_txt in msms_txt_list
    ]
    return pd.concat(psm_df_list, ignore_index=True)

train_df = load_psm_df(train_txt_list)
test_df = load_psm_df(test_txt_list)
test_df

,sequence,charge,rt,scan_num,raw_name,spec_idx,mods,mod_sites,nAA,rt_norm,precursor_mz
0,KLEEELR,3,44.996,34115,20181223_QX3_JoMu_SA_LC04_uPAC200cm_61_Deinoco...,34114,,,7,0.250033,306.174792
1,ALAGVQR,2,38.669,26563,20181223_QX3_JoMu_SA_LC04_uPAC200cm_61_Deinoco...,26562,,,7,0.214876,357.716488
2,SVLFLNK,2,113.480,112715,20181223_QX3_JoMu_SA_LC04_uPAC200cm_61_Deinoco...,112714,,,7,0.630585,410.749996
3,SVLFLNK,2,113.110,112266,20181223_QX3_JoMu_SA_LC04_uPAC200cm_61_Deinoco...,112265,,,7,0.628529,410.749996
4,SVEALNK,2,35.964,23307,20181223_QX3_JoMu_SA_LC04_uPAC200cm_61_Deinoco...,23306,,,7,0.199844,380.713611
...,...,...,...,...,...,...,...,...,...,...,...
959520,GQFALIPVSIFLQQELNAAEDVVVDQDEETITPSEVGGEQK,4,175.880,199212,20181010_QX1_JoMu_SA_Easy12-7_uPAC_500ng_2_Bac...,199211,,,41,0.977491,1111.557698
959521,WLEGGMVEITASYPAGVIGTTLENLQEAAAGEHEEWSLDYPK,3,175.230,198432,20181010_QX1_JoMu_SA_Easy12-7_uPAC_500ng_2_Bac...,198431,,,42,0.973879,1521.395119
959522,WLEGGMVEITASYPAGVIGTTLENLQEAAAGEHEEWSLDYPK,4,174.940,198078,20181010_QX1_JoMu_SA_Easy12-7_uPAC_500ng_2_Bac...,198077,Oxidation@M,6,42,0.972267,1145.296887
959523,MGKPEDMEAEGESASADSGSTSAGGGYGAGAWNSNTAYGTTR,3,97.891,103685,20181010_QX1_JoMu_SA_Easy12-7_uPAC_500ng_2_Bac...,103684,,,42,0.544050,1362.571380


In [3]:
import numpy as np

min_charge = 1
max_charge = 6

def get_charge_array(
    charge_list, 
    min_charge,
    max_charge,
):
    charge_array = np.zeros(max_charge-min_charge+1)
    for charge in charge_list:
        if charge <= max_charge and charge >= min_charge:
            charge_array[charge-min_charge] = 1.0
    return charge_array

In [4]:
train_charge_df = train_df.groupby("sequence")["charge"].apply(
    lambda x: get_charge_array(set(x), 
        min_charge=min_charge, 
        max_charge=max_charge
    )
).reset_index(drop=False).rename(columns={"charge":"charge_one_hot"})
train_charge_df

,sequence,charges
0,AAAAAAAAAAAAGDSAVNGQAEQQAIPTIGR,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0]"
1,AAAAAAAAAAAVAGTVPENETAEDK,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
2,AAAAAAAAAAEAAADDGANQR,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
3,AAAAAAAAAAGVWGATAEK,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0]"
4,AAAAAAAAAPSGPAPEGPAAK,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0]"
...,...,...
337362,YYYVHSYK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
337363,YYYVPMER,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
337364,YYYVQLSTGVSQWETPTDPAPVGATPGAAHEHPYGVPGSDADR,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
337365,YYYVVGEGTSMR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"


In [5]:
test_charge_df = test_df.groupby("sequence")["charge"].apply(
    lambda x: get_charge_array(set(x), 
        min_charge=min_charge, 
        max_charge=max_charge
    )
).reset_index(drop=False).rename(columns={"charge":"charge_one_hot"})
test_charge_df

,sequence,charges
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAMEEDSEASSSR,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0]"
1,AAAAAAAAAAGAAGGR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
2,AAAAAAAAADEQDEEK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
3,AAAAAAAAADEQDEEQEEEEAEEEEK,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
4,AAAAAAAAAGGAK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...
316208,YYYVQNVYTPVDENVYPDHR,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
316209,YYYVTEGLK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
316210,YYYWLDDGGK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
316211,YYYYGLGQDLDVGK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"


In [17]:
from peptdeep.model.model_shop import (
    Model_for_Generic_ModAASeq_BinaryClassification_Transformer,
    ModelInterface_for_Generic_ModAASeq_BinaryClassification,
    Model_for_Generic_AASeq_BinaryClassification_Transformer,
    ModelInterface_for_Generic_AASeq_BinaryClassification,
)
import torch

class ModelInterface_Chargeability(
    ModelInterface_for_Generic_AASeq_BinaryClassification
):
    def __init__(self,
        min_charge = 1,
        max_charge = 6,
    ):
        super().__init__(
            model_class=Model_for_Generic_AASeq_BinaryClassification_Transformer,
            nlayers=4, hidden_dim=256, dropout=0.1,
            output_dim=6, # six target values
        )
        self.num_target_values = 2
        self.target_column_to_train = 'charge_one_hot'
        self.target_column_to_predict = 'charge_probs'
        self.min_charge = min_charge
        self.max_charge = max_charge

    def _get_targets_from_batch_df(self, batch_df, **kwargs):
        return self._as_tensor(
            np.stack(batch_df[self.target_column_to_train].values), 
            dtype=torch.float32
        )

    def _prepare_predict_data_df(self, precursor_df, **kwargs):
        precursor_df[self.target_column_to_predict] = [
            [0]*self.num_target_values
        ]*len(precursor_df)
        self.predict_df = precursor_df

    def _set_batch_predict_data(self, batch_df, predict_values, **kwargs):
        if self._predict_in_order:
            self.predict_df.loc[:,self.target_column_to_predict].values[
                batch_df.index.values[0]:batch_df.index.values[-1]+1
            ] = list(predict_values)
        else:
            self.predict_df.loc[
                batch_df.index,self.target_column_to_predict
            ] = list(predict_values)

In [10]:
model = ModelInterface_Chargeability(
    min_charge=min_charge,
    max_charge=max_charge,
)
model.train(
    train_charge_df, epoch=10, warmup_epoch=3,
    verbose=True, verbose_each_epoch=True
)

2024-01-05 17:09:33> Training with fixed sequence length: 0


Epoch=1, nAA=16, batch=355, loss=0.6632: 100%|██████████| 42/42 [09:51<00:00, 14.08s/it]


[Training] Epoch=1, lr=3.3333333333333335e-05, loss=0.6848840025109304


Epoch=2, nAA=43, batch=355, loss=0.5035: 100%|██████████| 42/42 [10:01<00:00, 14.31s/it]


[Training] Epoch=2, lr=6.666666666666667e-05, loss=0.26914014266410347


...

[Training] Epoch=8, lr=1.8825509907063327e-05, loss=0.1095070582302943


Epoch=9, nAA=35, batch=355, loss=0.1987: 100%|██████████| 42/42 [10:17<00:00, 14.69s/it]


[Training] Epoch=9, lr=4.951556604879048e-06, loss=0.10474472552878966


Epoch=10, nAA=25, batch=355, loss=0.1447: 100%|██████████| 42/42 [10:16<00:00, 14.67s/it]

[Training] Epoch=10, lr=0.0, loss=0.10311434644759751


In [19]:
test_charge_df = model.predict(test_charge_df)
test_charge_df

,sequence,charges,nAA,charges_pred,accuracy,charge_probs
0,YYYYMWK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.015312695, 0.9958912, 0.006314218, 0.002732..."
1,NSFLLNK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.019172765, 0.9957242, 0.0054541924, 0.00318..."
2,ATWPSWK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.020584406, 0.9958437, 0.0053272, 0.00337232..."
3,IAPDVLR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.01129655, 0.99576086, 0.0077653183, 0.00215..."
4,ATVWMLR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.012776607, 0.99585164, 0.0071252496, 0.0023..."
...,...,...,...,...,...,...
316208,SAAPAPISASCPEPPMGSAVPTSSASIPVTSAVGDPGVGSVSPASPK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",47,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.012382595, 0.023153143, 0.33663192, 0.91722..."
316209,QAQSETPTPAASETPSPTPATVTPPAIDAQAAPAAPAAEAPVAPPEK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",47,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.009464889, 0.019967733, 0.25784796, 0.94688..."
316210,GHGEGDGAGASGAPGPQVAEAAAGADAVGEDDYQAYYLSAASEEGAER,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",48,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.010785331, 0.02513866, 0.14083713, 0.939656..."
316211,EPPAAPAAGAEATGDAAASATAAAGTTGEEGASTEQAPLLQEEAPSGVK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",49,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.010255453, 0.025947522, 0.2260452, 0.949546..."


In [20]:
def convert_probs_to_one_hot(test_charge_df, prob=0.5):
    test_charge_df["charge_pred"] = test_charge_df["charge_probs"].apply(
        lambda x: (x>prob).astype(float)
    )
    return test_charge_df

def test(test_charge_df:pd.DataFrame, prob):
    test_charge_df = convert_probs_to_one_hot(test_charge_df, prob)
    test_charge_df["accuracy"] = test_charge_df[["charge_one_hot","charge_pred"]].apply(
        lambda x: np.mean(x[0]==x[1]), axis=1
    )
    return test_charge_df

test_charge_df = test(test_charge_df, 0.5)
test_charge_df

,sequence,charges,nAA,charges_pred,accuracy,charge_probs
0,YYYYMWK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.015312695, 0.9958912, 0.006314218, 0.002732..."
1,NSFLLNK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.019172765, 0.9957242, 0.0054541924, 0.00318..."
2,ATWPSWK,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.020584406, 0.9958437, 0.0053272, 0.00337232..."
3,IAPDVLR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.01129655, 0.99576086, 0.0077653183, 0.00215..."
4,ATVWMLR,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",7,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",1.0,"[0.012776607, 0.99585164, 0.0071252496, 0.0023..."
...,...,...,...,...,...,...
316208,SAAPAPISASCPEPPMGSAVPTSSASIPVTSAVGDPGVGSVSPASPK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",47,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.012382595, 0.023153143, 0.33663192, 0.91722..."
316209,QAQSETPTPAASETPSPTPATVTPPAIDAQAAPAAPAAEAPVAPPEK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",47,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.009464889, 0.019967733, 0.25784796, 0.94688..."
316210,GHGEGDGAGASGAPGPQVAEAAAGADAVGEDDYQAYYLSAASEEGAER,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",48,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.010785331, 0.02513866, 0.14083713, 0.939656..."
316211,EPPAAPAAGAEATGDAAASATAAAGTTGEEGASTEQAPLLQEEAPSGVK,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",49,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",1.0,"[0.010255453, 0.025947522, 0.2260452, 0.949546..."


In [16]:
test_charge_df.accuracy.mean()

0.9594039460743231

In [23]:
test_charge_df.query("accuracy<=2.0/6")

,sequence,charges,nAA,charges_pred,accuracy,charge_probs
95051,FDDDDHHHHR,"[0.0, 1.0, 1.0, 0.0, 0.0, 0.0]",10,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0]",0.333333,"[0.01353842, 0.045806393, 0.063310586, 0.87132..."


### Predict spectral library based on charge prediction

In [30]:
def add_charge(df, model, prob=0.5):
    df["charge"] = df.charge_probs.apply(
        lambda x: np.arange(model.min_charge, model.max_charge+1)[
            x>prob
        ]
    )
    df = df.explode("charge")
    df.drop(columns="charge_probs", inplace=True)
    df["charge"] = df.charge.astype(np.int8)
    return df

In [31]:
from peptdeep.protein.fasta import PredictSpecLibFasta
fasta_lib = PredictSpecLibFasta(
    charged_frag_types=["b_z1","b_z2","y_z1","y_z2"],
    precursor_charge_min=0,
    precursor_charge_max=0,
    precursor_mz_min=400,
    precursor_mz_max=1000,
    peptide_length_min=7,
    peptide_length_max=30,
    max_missed_cleavages=2,
    var_mods=["Acetyl@Protein_N-term", "Oxidation@M"],
    fix_mods=["Carbamidomethyl@C"],
    special_mods=["Phospho@S","Phospho@T","Phospho@Y"],
    min_special_mod_num=0,
    max_special_mod_num=1,
)
fasta_lib.get_peptides_from_fasta(
    ["../test_data/small_fasta/test_protein.fasta"]
)
model.predict(fasta_lib.precursor_df)
fasta_lib._precursor_df = add_charge(
    fasta_lib.precursor_df, 
    model, prob=0.5
)
fasta_lib.append_decoy_sequence()
fasta_lib.add_modifications()
fasta_lib.add_special_modifications()
fasta_lib.precursor_df

,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,charge
0,QKELDSK,0,1,False,False,Phospho@S,6,7,2
1,QKELDSK,0,1,False,False,,,7,2
2,RKEVVHK,0,2,False,False,,,7,4
3,ILENAQR,0,0,False,False,,,7,2
4,ELDSKVR,0,1,False,False,Phospho@S,4,7,3
...,...,...,...,...,...,...,...,...,...
442,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@T,14,30,5
443,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@S,22,30,5
444,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@Y,28,30,5
445,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@S,29,30,5


In [32]:
fasta_lib.predict_all(
    predict_items=["rt","ms2"]
)
fasta_lib.precursor_df

2024-01-05 22:01:04> Predicting RT/IM/MS2 for 369 precursors ...
2024-01-05 22:01:04> Predicting RT ...


100%|██████████| 24/24 [00:00<00:00, 79.09it/s]


2024-01-05 22:01:05> Predicting MS2 ...


100%|██████████| 24/24 [00:00<00:00, 39.67it/s]

2024-01-05 22:01:05> End predicting RT/IM/MS2


,sequence,protein_idxes,miss_cleavage,is_prot_nterm,is_prot_cterm,mods,mod_sites,nAA,charge,precursor_mz,rt_pred,rt_norm_pred,nce,instrument,frag_start_idx,frag_stop_idx
0,QKELDSK,0,1,False,False,Phospho@S,6,7,2,464.212790,0.009475,0.009475,30.0,Lumos,0,6
1,QKELDSK,0,1,False,False,,,7,2,424.229625,0.012003,0.012003,30.0,Lumos,6,12
2,ILENAQR,0,0,False,False,,,7,2,422.237784,0.106143,0.106143,30.0,Lumos,12,18
3,QEQLLLK,0,0,False,False,,,7,2,436.266010,0.372224,0.372224,30.0,Lumos,18,24
4,CKTLQNR,0,1,False,False,Carbamidomethyl@C;Phospho@T,1;3,7,2,500.225709,0.005521,0.005521,30.0,Lumos,24,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@T,14,30,5,732.333707,0.900869,0.900869,30.0,Lumos,6596,6625
365,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@S,22,30,5,732.333707,0.906644,0.906644,30.0,Lumos,6625,6654
366,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@Y,28,30,5,732.333707,0.904113,0.904113,30.0,Lumos,6654,6683
367,QDWEHAANDVSFATIRFHDLLSQLDDQYSR,0,1,False,False,Phospho@S,29,30,5,732.333707,0.899686,0.899686,30.0,Lumos,6683,6712


In [33]:
fasta_lib.save_hdf("../test_data/speclib/predicted_charged.speclib.hdf")